### Poynting vector of an oscillating electric dipole

Before beginning, we need to identify the basis vectors when we switch to spherical coordinates

In [1]:
# Importing sympy.vector module and useful tools
from sympy.vector import *
from sympy import pi, cos, sin, symbols, diff, integrate, Function, Matrix, Eq, solve, solve_linear_system
from sympy.abc import a, b, c
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Cartesin system of coordinate
C = CoordSys3D(" ", transformation = "cartesian", vector_names = ("i", "j", "k"))
i, j, k = (C.i, C.j, C.k)

In [3]:
# Defining rho, theta, phi as time dependent variables
t = symbols("t", real = True, nonnegative = True)
rho = Function("rho", real = True, nonnegative = True)(t)
theta = Function("theta", real = True, nonnegative = True)(t)
phi = Function("phi", real = True, nonnegative = True)(t)

In [4]:
# Position of a particle in spherical coordinates
x = i * rho * cos(phi) * sin(theta) + j * rho * sin(phi) * sin(theta) + k * rho * cos(theta)
x

(rho(t)*sin(theta(t))*cos(phi(t)))* .i + (rho(t)*sin(phi(t))*sin(theta(t)))* .j + (rho(t)*cos(theta(t)))* .k

In [5]:
# Velocity of a particle in spherical coordinates
diff(x, t, 1)

(-rho(t)*sin(phi(t))*sin(theta(t))*Derivative(phi(t), t) + rho(t)*cos(phi(t))*cos(theta(t))*Derivative(theta(t), t) + sin(theta(t))*cos(phi(t))*Derivative(rho(t), t))* .i + (rho(t)*sin(phi(t))*cos(theta(t))*Derivative(theta(t), t) + rho(t)*sin(theta(t))*cos(phi(t))*Derivative(phi(t), t) + sin(phi(t))*sin(theta(t))*Derivative(rho(t), t))* .j + (-rho(t)*sin(theta(t))*Derivative(theta(t), t) + cos(theta(t))*Derivative(rho(t), t))* .k

Deriving the velocity of a particle in spherical coordinates, we got:

\begin{equation*}
    \vec{v} = \dot{\rho} \, \hat{e_r} + \rho \dot{\theta} \, \hat{e_{\theta}} + \rho \dot{\phi} \sin(\theta) \, \hat{e_{\phi}}
\end{equation*}

Then we found the basis vectors

In [6]:
# r, theta, phi as independent variables
rho, theta, phi = symbols("rho, theta, phi", real = True, nonnegative = True)

In [7]:
# Radial unit vector
e_r = Matrix([cos(phi) * sin(theta),
              sin(phi) * sin(theta),
              cos(theta)])
e_r

Matrix([
[sin(theta)*cos(phi)],
[sin(phi)*sin(theta)],
[         cos(theta)]])

In [8]:
# Tangential unit vector
e_theta = Matrix([cos(phi) * cos(theta),
                  sin(phi) * cos(theta),
                  - sin(theta)])
e_theta

Matrix([
[cos(phi)*cos(theta)],
[sin(phi)*cos(theta)],
[        -sin(theta)]])

In [9]:
# Azimuthal unit vector
e_phi = Matrix([- sin(phi),
                cos(phi),
                0])
e_phi

Matrix([
[-sin(phi)],
[ cos(phi)],
[        0]])

We've to express the basis vector $\hat{k} = (0, 0, 1)$ in terms of the spherical basis vectors:

\begin{equation*}
    \hat{k} = \alpha \, \hat{e_r} + \beta \, \hat{e_{\theta}} + \gamma \, \hat{e_{\phi}}  
\end{equation*}

In [10]:
# k unit vector
k_matrix = Matrix([0,
                   0,
                   1])
k_matrix

Matrix([
[0],
[0],
[1]])

In [11]:
# Defining an augmented matrix
M = Matrix.hstack(e_r, e_theta, e_phi).row_join(k_matrix)
M

Matrix([
[sin(theta)*cos(phi), cos(phi)*cos(theta), -sin(phi), 0],
[sin(phi)*sin(theta), sin(phi)*cos(theta),  cos(phi), 0],
[         cos(theta),         -sin(theta),         0, 1]])

In [12]:
# Solving for alpha, beta, gamma
sol = solve_linear_system(M, a, b, c)
sol

{a: cos(theta)/(sin(theta)**2 + cos(theta)**2),
 b: -sin(theta)/(sin(theta)**2 + cos(theta)**2),
 c: 0}

In [13]:
# alpha
sol[a].trigsimp()

cos(theta)

In [14]:
# beta
sol[b].trigsimp()

-sin(theta)

Then we got: $\hat{k} = \cos(\theta) \, \hat{e_r} - \sin{(\theta)} \, \hat{e_{\theta}}$. We will make use of this result soon. $\\$
Another quantity we need is the jacobian when converting from the cartesian to spherical coordinate system

In [15]:
# Position of a point in spherical coordinates
x = i * rho * cos(phi) * sin(theta) + j * rho * sin(phi) * sin(theta) + k * rho * cos(theta)
x

(rho*sin(theta)*cos(phi))* .i + (rho*sin(phi)*sin(theta))* .j + (rho*cos(theta))* .k

In [16]:
# Computing J
J = Matrix([[diff(x.components[i], rho), diff(x.components[i], theta), diff(x.components[i], phi)],
            [diff(x.components[j], rho), diff(x.components[j], theta), diff(x.components[j], phi)],
            [diff(x.components[k], rho), diff(x.components[k], theta), diff(x.components[k], phi)]])
J

Matrix([
[sin(theta)*cos(phi), rho*cos(phi)*cos(theta), -rho*sin(phi)*sin(theta)],
[sin(phi)*sin(theta), rho*sin(phi)*cos(theta),  rho*sin(theta)*cos(phi)],
[         cos(theta),         -rho*sin(theta),                        0]])

In [17]:
# jacobian
J.det().trigsimp()

rho**2*sin(theta)

In [18]:
# Spherical system of coordinate
S = CoordSys3D(" ", transformation = "spherical", vector_names = ("e_r", "e_t", "e_p"), variable_names = ("r", "theta", "phi"))
e_r, e_theta, e_phi = (S.e_r, S.e_t, S.e_p)
r, theta, phi = (S.r, S.theta, S.phi)

In [19]:
# Defining constants
p0, u0, e0, c, omega = symbols("p_0, u_0, e_0, c, omega", real = True, positive = True, constant = True)

The potential of an oscillating electric dipole is approximated as:

\begin{equation*}
    \Phi (r, \theta, t) = - \frac{p_0 \omega }{4 \pi \varepsilon _0 c} \, \cos(\theta / r) \, \sin(\omega (t - r / c))
\end{equation*}

In [20]:
# Defining the potential
potential = - (p0 * omega) / (4 * pi * e0 * c) * cos(theta / r) * sin(omega * (t - r / c))
potential

-omega*p_0*sin(omega*(- .r/c + t))*cos( .theta/ .r)/(4*pi*c*e_0)

And the potential vector is approximated as well:

\begin{equation*}
    \vec{A}(r, \theta, t) = - \frac{u_0 p_0 \omega}{4 \pi r} \sin(\omega (t - r / c)) \, \hat{k}
\end{equation*}

By expressing $\hat{k}$ in terms of the spherical basis vectors, we obtain:

\begin{equation*}
    \vec{A}(r, \theta, t) = - \frac{\mu _0 p_0 \omega}{4 \pi r} \sin(\omega (t - r / c)) (\cos(\theta) \, \hat{e_r} - \sin(\theta) \, \hat{e_{\theta}})
\end{equation*}

In [21]:
# Defining the potential vector 
A = - (u0 * p0 * omega) / (4 * pi * r) * sin(omega * (t - r / c)) * (cos(theta) * e_r - sin(theta) * e_theta)
A

(-omega*p_0*u_0*sin(omega*(- .r/c + t))*cos( .theta)/(4* .r*pi))* .e_r + (omega*p_0*u_0*sin( .theta)*sin(omega*(- .r/c + t))/(4* .r*pi))* .e_t

Let's compute the electric field: $\vec{E} = - \vec{\nabla} \Phi - \partial \vec{A} / \partial t$

In [22]:
# Computing the electric field
E = (- gradient(potential) - diff(A, t, 1)).simplify()
E

(omega*p_0*(- .r**2*omega*cos( .theta/ .r)*cos(omega*( .r - c*t)/c) +  .r*c**2*e_0*omega*u_0*cos( .theta)*cos(omega*( .r - c*t)/c) -  .theta*c*sin( .theta/ .r)*sin(omega*( .r - c*t)/c))/(4* .r**2*pi*c**2*e_0))* .e_r + (omega*p_0*(- .r*c*e_0*omega*u_0*sin( .theta)*cos(omega*( .r - c*t)/c) + sin( .theta/ .r)*sin(omega*( .r - c*t)/c))/(4* .r**2*pi*c*e_0))* .e_t

And the magnetic field as well: $\vec{B} = \vec{\nabla} \times \vec{A}$

In [23]:
# Computing the magnetic field
B = curl(A).simplify()
B

(omega*p_0*u_0*(- .r*omega*cos(omega*( .r - c*t)/c) + c*sin(omega*( .r - c*t)/c))*sin( .theta)/(4* .r**2*pi*c))* .e_p

Then we compute the Poynting vector: $\vec{S} = 1 / \mu _0 \, \vec{E} \times \vec{B}$

In [24]:
# Computing the Poynting vector
S = (1 / u0 * (E ^ B)).simplify()
S

(omega**2*p_0**2*( .r*omega*cos(omega*( .r - c*t)/c) - c*sin(omega*( .r - c*t)/c))*( .r*c*e_0*omega*u_0*sin( .theta)*cos(omega*( .r - c*t)/c) - sin( .theta/ .r)*sin(omega*( .r - c*t)/c))*sin( .theta)/(16* .r**4*pi**2*c**2*e_0))* .e_r + (-omega**2*p_0**2*( .r*omega*cos(omega*( .r - c*t)/c) - c*sin(omega*( .r - c*t)/c))*( .r**2*omega*cos( .theta/ .r)*cos(omega*( .r - c*t)/c) -  .r*c**2*e_0*omega*u_0*cos( .theta)*cos(omega*( .r - c*t)/c) +  .theta*c*sin( .theta/ .r)*sin(omega*( .r - c*t)/c))*sin( .theta)/(16* .r**4*pi**2*c**3*e_0))* .e_t

The Poynting vector thus has both a radial and a tangential component. Only the radial component is of interest to us, as it is the one that actually transports energy through space. Then let's expand it

In [25]:
# Poynting vector's radial component
S.components[e_r].expand()

omega**4*p_0**2*u_0*sin( .theta)**2*cos( .r*omega/c - omega*t)**2/(16* .r**2*pi**2*c) - omega**3*p_0**2*u_0*sin( .theta)**2*sin( .r*omega/c - omega*t)*cos( .r*omega/c - omega*t)/(16* .r**3*pi**2) - omega**3*p_0**2*sin( .theta)*sin( .theta/ .r)*sin( .r*omega/c - omega*t)*cos( .r*omega/c - omega*t)/(16* .r**3*pi**2*c**2*e_0) + omega**2*p_0**2*sin( .theta)*sin( .theta/ .r)*sin( .r*omega/c - omega*t)**2/(16* .r**4*pi**2*c*e_0)

In [26]:
# Keeping only the first term
S = S.components[e_r].expand().args[1]
S

omega**4*p_0**2*u_0*sin( .theta)**2*cos( .r*omega/c - omega*t)**2/(16* .r**2*pi**2*c)

The flux of the Poynting vector through a closed surface, with its sign reversed, represents the total amount of electromagnetic energy contained within the volume enclosed by the surface per unit time:

\begin{equation*}
    \Phi _ {\partial V} (\vec{S}) = - \frac{\partial U}{\partial t}
\end{equation*}

Then, let's compute the flux of $\vec{S}$ through the surface of a sphere with radius $a$

In [27]:
# Computing the flux of S
dUdt = - integrate(S.subs(r, a) * a ** 2 * sin(theta), (theta, 0, pi), (phi, 0, 2 * pi))
dUdt

-omega**4*p_0**2*u_0*cos(a*omega/c - omega*t)**2/(6*pi*c)

And finally we compute the average power over a period

In [28]:
(omega / (2 * pi) * integrate(dUdt, (t, 0, 2 * pi / omega))).trigsimp()

-omega**4*p_0**2*u_0/(12*pi*c)